# **Koneksi Ke Google Big Query**

In [1]:
from google.colab import auth
from google.colab import data_table
from google.cloud import bigquery
from pandas_gbq import to_gbq
import pandas as pd
import numpy as np

# PRoses autentikasi akun
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
from google.cloud import bigquery

# Buat BigQuery client
project_id = 'certain-cursor-468913-c0'
bq_client = bigquery.Client(project = project_id)

## **Proses Ekstraksi Data dari sumber (Extract)**

In [45]:
%%bigquery data_transaksi --project {project_id}

-- Tulis Query disini
SELECT * FROM dqlab-468906.dqcommerce.tbl_transaction

Query is running:   0%|          |

Downloading:   0%|          |

Setelah query dijalankan maka akan disimpan sebagai dataframe pandas

In [17]:
%%bigquery data_product --project {project_id}

-- Tulis Query disini
SELECT * FROM dqlab-468906.dqcommerce.tbl_product

Query is running:   0%|          |

Downloading:   0%|          |

In [18]:
%%bigquery data_funnel --project {project_id}

-- Tulis Query disini
SELECT * FROM dqlab-468906.dqcommerce.tbl_funnel

Query is running:   0%|          |

Downloading:   0%|          |

## **Periksa Informasi Umum pada Data**

In [20]:
# Periksa info umum pada data
data_transaksi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829262 entries, 0 to 829261
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   trx_id      829257 non-null  object
 1   product_id  829262 non-null  object
 2   trx_date    829262 non-null  Int64 
 3   trx_time    829262 non-null  dbtime
 4   units       829251 non-null  Int64 
dtypes: Int64(2), dbtime(1), object(2)
memory usage: 33.2+ MB


In [21]:
# Periksa info umum pada data
data_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        37 non-null     object
 1   product_name      37 non-null     object
 2   product_category  37 non-null     object
 3   product_cost      37 non-null     object
 4   product_price     37 non-null     object
dtypes: object(5)
memory usage: 1.6+ KB


In [22]:
# Periksa info umum pada data
data_funnel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15950 entries, 0 to 15949
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         15950 non-null  Int64 
 1   product_id   15950 non-null  object
 2   purchase     15950 non-null  Int64 
 3   add_to_cart  15950 non-null  Int64 
 4   click        15950 non-null  Int64 
 5   view         15950 non-null  Int64 
dtypes: Int64(5), object(1)
memory usage: 825.7+ KB


## **Proses Transformasi Data (Transform)**

In [46]:
# Konversi kolom trx_date dari Integer menjadi string
data_transaksi['trx_date'] = data_transaksi['trx_date'].astype('str')
data_transaksi['trx_date'] = np.where(data_transaksi['trx_date'].str.len() == 7, '0' + data_transaksi['trx_date'], data_transaksi['trx_date'])
data_transaksi.sample(10)

,trx_id,product_id,trx_date,trx_time,units
521623,DQTrx019096,DQProduk-025,21012024,21:23:36,1
626235,DQTrx631339,DQProduk-030,19052025,02:26:32,1
160028,DQTrx606129,DQProduk-006,04052025,13:06:41,1
106014,DQTrx821437,DQProduk-003,26092025,17:20:58,1
269488,DQTrx704831,DQProduk-009,05072025,01:52:28,1
782407,DQTrx378246,DQProduk-018,04122024,23:33:37,3
110389,DQTrx813143,DQProduk-004,20092025,05:01:35,1
210718,DQTrx208784,DQProduk-008,10072024,04:57:42,1
650867,DQTrx365061,DQProduk-031,25112024,04:55:53,1
22592,DQTrx302299,DQProduk-001,06102024,04:35:43,1


In [47]:
# Buat kondisi
data_transaksi['trx_date'] = pd.to_datetime(data_transaksi['trx_date'], format = '%d%m%Y')
data_transaksi.sample(10)

,trx_id,product_id,trx_date,trx_time,units
97796,DQTrx673964,DQProduk-003,2025-06-16,19:43:52,1
336944,DQTrx376000,DQProduk-014,2024-12-02,18:48:26,1
352174,DQTrx257706,DQProduk-015,2024-08-25,15:07:12,1
75351,DQTrx324394,DQProduk-003,2024-10-24,18:10:01,1
16503,DQTrx200514,DQProduk-001,2024-07-02,14:39:30,1
650539,DQTrx351723,DQProduk-031,2024-11-15,02:05:56,1
278706,DQTrx147150,DQProduk-010,2024-05-19,04:42:50,1
582678,DQTrx316728,DQProduk-027,2024-10-18,11:55:26,1
547878,DQTrx497917,DQProduk-025,2025-02-24,07:19:37,1
239909,DQTrx617832,DQProduk-008,2025-05-11,23:47:46,1


In [48]:
data_transaksi.isna().sum()

,0
trx_id,5
product_id,0
trx_date,0
trx_time,0
units,11


In [49]:
# data asli sebelum di hapus/drop
data_transaksi.shape

(829262, 5)

In [51]:
# hapus data yang kosong
data_transaksi = data_transaksi[data_transaksi['trx_id'].notnull()]
data_transaksi.shape

(829257, 5)

In [52]:
data_transaksi.isna().sum()

,0
trx_id,0
product_id,0
trx_date,0
trx_time,0
units,11


In [54]:
data_transaksi[data_transaksi['units'].isna()]

,trx_id,product_id,trx_date,trx_time,units
0,DQTrx829214,DQProduk-016,2025-09-30,01:18:05,<NA>
1,DQTrx829166,DQProduk-017,2025-09-30,18:58:33,<NA>
2,DQTrx829118,DQProduk-018,2025-09-30,06:58:09,<NA>
3,DQTrx829186,DQProduk-019,2025-09-30,09:28:13,<NA>
4,DQTrx829212,DQProduk-019,2025-09-30,15:00:55,<NA>
5,DQTrx829213,DQProduk-019,2025-09-30,16:06:16,<NA>
6,DQTrx829235,DQProduk-019,2025-09-30,17:49:03,<NA>
7,DQTrx829167,DQProduk-024,2025-09-30,21:51:24,<NA>
8,DQTrx829168,DQProduk-025,2025-09-30,02:05:10,<NA>
9,DQTrx829254,DQProduk-025,2025-09-30,08:12:00,<NA>


In [56]:
# data transaksi kolom unit null diubah menjadi 0
data_transaksi['units'] = data_transaksi['units'].fillna(0)

In [57]:
data_transaksi[data_transaksi['units'] == 0]

,trx_id,product_id,trx_date,trx_time,units
0,DQTrx829214,DQProduk-016,2025-09-30,01:18:05,0
1,DQTrx829166,DQProduk-017,2025-09-30,18:58:33,0
2,DQTrx829118,DQProduk-018,2025-09-30,06:58:09,0
3,DQTrx829186,DQProduk-019,2025-09-30,09:28:13,0
4,DQTrx829212,DQProduk-019,2025-09-30,15:00:55,0
5,DQTrx829213,DQProduk-019,2025-09-30,16:06:16,0
6,DQTrx829235,DQProduk-019,2025-09-30,17:49:03,0
7,DQTrx829167,DQProduk-024,2025-09-30,21:51:24,0
8,DQTrx829168,DQProduk-025,2025-09-30,02:05:10,0
9,DQTrx829254,DQProduk-025,2025-09-30,08:12:00,0


In [58]:
# pengecekan nilai null pada units
data_transaksi[data_transaksi['units'].isna()]

,trx_id,product_id,trx_date,trx_time,units


In [64]:
# cek duplikat
data_transaksi['trx_id'].duplicated().sum()

np.int64(0)

In [65]:
data_transaksi.duplicated().sum()

np.int64(0)

In [60]:
data_product.head()

,product_id,product_name,product_category,product_cost,product_price
0,DQProduk-009,Pulpen Metal DQLab,Alat Tulis,IDR 149850,IDR 164850
1,DQProduk-003,Sticky Notes DQLab,Alat Tulis,IDR 29850,IDR 59850
2,DQProduk-025,Tipe-X Roller DQLab,Alat Tulis,IDR 29850,IDR 44850
3,DQProduk-016,Set Pulpen DQLab,Alat Tulis,IDR 44850,IDR 149850
4,DQProduk-028,Penanda Warna DQLab,Alat Tulis,IDR 59850,IDR 164850


In [62]:
# penghapusan IDR
for col in ['product_cost', 'product_price']:
  data_product[col] = data_product[col].str.replace('IDR ', '')
  # diubah ke int
  data_product[col] = data_product[col].astype('int64')

In [63]:
data_product.head()

,product_id,product_name,product_category,product_cost,product_price
0,DQProduk-009,Pulpen Metal DQLab,Alat Tulis,149850,164850
1,DQProduk-003,Sticky Notes DQLab,Alat Tulis,29850,59850
2,DQProduk-025,Tipe-X Roller DQLab,Alat Tulis,29850,44850
3,DQProduk-016,Set Pulpen DQLab,Alat Tulis,44850,149850
4,DQProduk-028,Penanda Warna DQLab,Alat Tulis,59850,164850


In [66]:
# pengecekan duplikat
data_product.duplicated().sum()

np.int64(2)

In [71]:
data_product[data_product.duplicated(subset='product_id', keep=False)]

,product_id,product_name,product_category,product_cost,product_price
17,DQProduk-035,Kartu Ucapan DQLab,Kertas & Cetak,59850,119850
18,DQProduk-035,Kartu Ucapan DQLab,Kertas & Cetak,59850,119850
27,DQProduk-015,Mousepad Custom DQLab,Merchandise,59850,89850
28,DQProduk-015,Mousepad Custom DQLab,Merchandise,59850,89850


In [72]:
data_product.shape

(37, 5)

In [73]:
data_product = data_product.drop_duplicates(subset='product_id', keep='first')

In [74]:
data_product.shape

(35, 5)

In [75]:
data_integration = data_transaksi.merge(data_product, on = 'product_id', how = 'left')
data_integration.head()

,trx_id,product_id,trx_date,trx_time,units,product_name,product_category,product_cost,product_price
0,DQTrx829214,DQProduk-016,2025-09-30,01:18:05,0,Set Pulpen DQLab,Alat Tulis,44850,149850
1,DQTrx829166,DQProduk-017,2025-09-30,18:58:33,0,Set Tulis Eksekutif DQLab,Merchandise,209850,299850
2,DQTrx829118,DQProduk-018,2025-09-30,06:58:09,0,Printer Mini DQLab,Elektronik,524850,599850
3,DQTrx829186,DQProduk-019,2025-09-30,09:28:13,0,Kertas Warna DQLab,Kertas & Cetak,209850,239850
4,DQTrx829212,DQProduk-019,2025-09-30,15:00:55,0,Kertas Warna DQLab,Kertas & Cetak,209850,239850


## **Proses Simpan Data (Load)**

In [83]:
import pandas as pd

# Upload ke BigQuery - tabel belum ada
to_gbq(
    data_integration,
    destination_table = 'tbl_dtw.tbl_integration',
    project_id = project_id,
    if_exists = 'fail'
)

100%|██████████| 1/1 [00:00<00:00, 2395.38it/s]


---

### Overview Total Penjualan Periode Data Tiap Bulan

In [84]:
data_transaksi['month_year'] = data_transaksi['trx_date'].dt.strftime('%Y-%m')
data_transaksi.head()

,trx_id,product_id,trx_date,trx_time,units,month_year
0,DQTrx829214,DQProduk-016,2025-09-30,01:18:05,0,2025-09
1,DQTrx829166,DQProduk-017,2025-09-30,18:58:33,0,2025-09
2,DQTrx829118,DQProduk-018,2025-09-30,06:58:09,0,2025-09
3,DQTrx829186,DQProduk-019,2025-09-30,09:28:13,0,2025-09
4,DQTrx829212,DQProduk-019,2025-09-30,15:00:55,0,2025-09


In [85]:
penjualan_bulanan = data_transaksi.groupby(['month_year'], as_index = False).agg(total = ('units', 'sum'))
penjualan_bulanan.head()

,month_year,total
0,2024-01,38004
1,2024-02,36935
2,2024-03,39981
3,2024-04,47102
4,2024-05,46910


In [86]:
import plotly.express as px

# Membuat line chart
fig = px.line(
    penjualan_bulanan,
    x="month_year",
    y="total",
    title="<b>Grafik Total Penjualan Bulanan DQ-Commerce</b>",
    color_discrete_sequence = ['#39074E'],
    markers=False)

"""
fig.add_vrect(
    x0="2025-03", x1="2025-09",
    fillcolor="#E73c35",
    opacity=0.5,
    layer="below",
    line_width=0,
    #annotation_text="Tahun 2025",
    #annotation_position="top "
)
"""

fig.update_layout(
    height = 450,
    width = 1100,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    yaxis = dict(
        showline = True,
        linewidth = 1,
        color = '#544018',
        linecolor = '#544018',
        showgrid=False,
    ),
    xaxis = dict(
        title = '',
        showline = True,
        linewidth = 1,
        color = '#544018',
        linecolor = '#544018',
        showgrid=False,
    ),
)

fig.show()

In [87]:
penjualan_per_produk = data_transaksi.groupby(['month_year', 'product_id'], as_index = False).agg(total = ('units', 'sum'))
penjualan_per_produk.head()

,month_year,product_id,total
0,2024-01,DQProduk-001,3441
1,2024-01,DQProduk-002,1513
2,2024-01,DQProduk-003,76
3,2024-01,DQProduk-004,165
4,2024-01,DQProduk-005,276


In [88]:
import plotly.express as px

# Membuat line chart
fig = px.line(
    penjualan_per_produk, #[penjualan['month_year'] >= '2025-01'],
    x="month_year",
    y="total",
    color="product_id",
    title="<b>Grafik Total Penjualan Bulanan DQ-Commerce Per Produk</b>",
    markers=True
)

fig.update_layout(
    height = 550,
    width = 1200,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    yaxis = dict(
        showline = True,
        linewidth = 1,
        color = '#544018',
        linecolor = '#544018',
        showgrid=False,
    ),
    xaxis = dict(
        title = '',
        showline = True,
        linewidth = 1,
        color = '#544018',
        linecolor = '#544018',
        showgrid=False,
    ),
)

fig.show()

In [89]:
import plotly.graph_objects as go

highlight_product = ['DQProduk-003', 'DQProduk-019', 'DQProduk-025', 'DQProduk-018', 'DQProduk-031']

fig = go.Figure()

# Loop setiap produk
for product in penjualan_per_produk['product_id'].unique():
    df_product = penjualan_per_produk[penjualan_per_produk['product_id'] == product]

    # Tentukan properti styling
    if product in highlight_product:
        line_color = 'purple'
        line_width = 3
    else:
        line_color = 'lightgray'
        line_width = 1.2

    # Tambahkan trace
    fig.add_trace(go.Scatter(
        x=df_product['month_year'],
        y=df_product['total'],
        mode='lines+markers',
        name=product,
        line=dict(
            width=line_width,
            color=line_color
        )
    ))

# Layout styling
fig.update_layout(
    title="<b>Grafik Total Penjualan Bulanan DQ-Commerce Per Produk</b>",
    height=550,
    width=1200,
    plot_bgcolor='rgba(0, 0, 0, 0)',
    xaxis=dict(
        showline=True,
        linewidth=1,
        color='#544018',
        linecolor='#544018',
        title='',
        showgrid=False,
    ),
    yaxis=dict(
        showline=True,
        linewidth=1,
        color='#544018',
        linecolor='#544018',
        showgrid=False,
    )
)

fig.show()